<a href="https://colab.research.google.com/github/Gaurav7004/RSE_TOOL/blob/main/RSE_TOOL_17thMarch2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
# ! pip install pandarallel
# from pandarallel import pandarallel
import json
import re

In [2]:
# pandarallel.initialize()

In [3]:
# df = pd.read_csv('Data1.csv')

In [4]:
## Data is the dictionary
data = {}

## Keys
key = ''

## values
values = []

## Open
with open('Example1.RSE', "r") as f:
    for line in f:
        line = line.strip()
        if line.startswith('#'):
            if key:
                key = key.split("'")[0]
                key = key.split("\t")[0]
                data[key] = [v for v in values if v and not v.startswith("'")]
            key = line.split("'")[0]
            key = key.split("\t")[0]
            values = []
        elif line.startswith("'"):
            pass
        else:
            line = line.split("'")[0]
            values.append(line.strip())

# add last group
if key:
    data[key] = [v for v in values if v and not v.startswith("'")]

print(json.dumps(data, indent=4))

{
    "#SAMPLING SUBSAMPLE2": [],
    "#GROUP 2": [
        "SEC   C sec",
        "ST_GR C NEWVAR"
    ],
    "#FILTER 1": [
        "AGE>=15"
    ],
    "#VARIABLE 3  ": [
        "POP\tN  NEWVAR",
        "LF\tN  NEWVAR",
        "WRK\tN  NEWVAR"
    ],
    "#FILE Data1.csv  ": [],
    "#RENAME  5    ": [
        "STRMID C sec+st+strm+sstrm",
        "SS     C ss",
        "NSS    N  nss",
        "NSC    N  nsc",
        "MULT   N  mult"
    ],
    "#TRANSFORM 4  ": [
        "ST_GR=\"G1\"   in(ST,\"10,19,20,21\")",
        "ST_GR=\"G2\"   in(ST,\"11,12,13,14,15,16,17,18\")",
        "ST_GR=\"G3\"   Otherwise",
        "POP=1",
        "WRK=1  in(CWS,\"11,12,21,31,41,51,61,61,71,72\")",
        "WRK=0  NOT in(CWS,\"11,12,21,31,41,51,61,61,71,72\")",
        "LF=1      in(CWS,\"11,12,21,31,41,51,61,61,71,72,81\")",
        "LF=0      Otherwise"
    ],
    "#EST_RSE 7": [
        "S POP",
        "E POP",
        "R POP",
        "E 100*LF/POP",
        "R LF/POP",
        "E 100*WRK

In [84]:
# Open the text file for reading
with open('Example1.RSE', 'r') as input_file:
    # Read the contents of the text file
    file_contents = input_file.read()

# Split the file contents into lines
lines = file_contents.split('\n')

# Initialize some variables to hold the generated Python code
sampling_code = ''
filter_code = ''
variable_code = ''
rename_code = ''
transform_code = ''
est_rse_code = ''

# Loop through each line of the file contents and generate Python code
for line in lines:
    if line.startswith('#SAMPLING'):
        if '\t' in line:
            _, sampling_scheme = line.split('\t')
        else:
            _, sampling_scheme, _ = line.split()

    elif line.startswith('#FILTER'):
        for line in lines:
            if line.startswith('#SAMPLING'):
                # Generate code for the sampling scheme
                _, sampling_scheme = line.split(maxsplit=1)
                sampling_code = f'sampling_scheme = "{sampling_scheme.strip()}"'
            elif line.startswith('#FILTER'):
                # Extract the number of conditions from the line
                num_conditions_match = re.search(r'\d+', line)
                if num_conditions_match:
                    num_conditions = int(num_conditions_match.group())
                    filter_conditions = [lines[i] for i in range(1, num_conditions+1)]
                    filter_code = ' & '.join(filter_conditions)
                else:
                    filter_code = ''

    elif line.startswith('#VARIABLE'):
        # Generate code for the selected variables
        values = line.split()
        if len(values) == 2:
            _, num_variables = values
            variables = [lines[i].split() for i in range(1, int(num_variables)+1)]
            variable_code = ', '.join([f'{var[1]} = {var[0]}' for var in variables])


    elif line.startswith('#RENAME'):
        # Generate code for renaming variables
        parts = line.split('#')[0].strip().split()
        if len(parts) > 0:
            num_renames = parts[0]
            renames = [lines[i].strip().split() for i in range(1, int(num_renames)+1)]
            rename_code += '\n' + '\n'.join([f'{rename[0]} = {rename[1]}' for rename in renames])


    elif line.startswith('#TRANSFORM'):
        # Generate code for transforming variables
        num_transforms = line.strip().split()[1]
        transforms = [lines[i] for i in range(1, int(num_transforms)+1)]
        transform_code = '\n'.join(transforms)

    elif line.startswith('#EST_RSE'):
        # Generate code for generating columns
        parts = line.split()
        if len(parts) != 3:
            continue  # Skip line

        num_columns = parts[1]
        columns = [lines[i].split()[0] for i in range(2, int(num_columns)+2)]
        est_rse_code = '\n'.join([f'{col} = {sampling_scheme}({col}, {filter_code})' for col in columns])
        code_blocks.append(est_rse_code)



# Write the generated Python code to a file
with open('output.py', 'w') as output_file:
    output_file.write(f'from sampling import {sampling_scheme}\n\n')
    output_file.write(f'{sampling_code}\n')
    output_file.write(f'{filter_code}\n')
    output_file.write(f'{variable_code}\n')
    output_file.write(f'{rename_code}\n')
    output_file.write(f'{transform_code}\n')
    output_file.write(f'{est_rse_code}\n')


In [6]:
def SAMPLING_(data, df):
    for key, value in data.items():
        pass

def GROUP_(data, df):
    for key, value in data.items():
        ### To read the sampling method used
        if str('SAMPLING').lower() in str(key).lower():
            if str('SUBSAMPLE2').lower() in str(key).lower():
                return str('SUBSAMPLE2')
            elif str('SRSWR').lower() in str(key).lower():
                pass
            elif str('SRSWOR').lower() in str(key).lower():
                pass

def FILTER_(data, df):
    for key, value in data.items():
        ### To read the sampling method used
        if str('FILTER').lower() in str(key).lower():
            ### Replace logical AND '&&' with 'and'
            if '&&' in value[0]:
                return value[0].replace('&&', 'and')
            ### Replace logical OR '||' with 'or'
            elif '||' in value[0]:
                return value[0].replace('||', 'or')
            else:
                return value[0]

def FILE_(data):
    for key, value in data.items():
        ### To read the file used
        if str('FILE').lower() in str(key).lower():
            if key.strip().endswith('.xlsx') or key.strip().endswith('.csv'):
                match = re.search(r'[A-Za-z0-9]+\.xlsx|[A-Za-z0-9]+\.csv', key)   
                if match:
                    file_name = match.group(0)
                    return file_name
                else:
                    file_name = None
                    return file_name
                print(file_name)

            # return file_name

def RENAME_(data, df):
    # create an empty dictionary to store the column names
    column_dict = {}

    for key, value in data.items():
        ### To read the file used
        if str('RENAME').lower() in str(key).lower():
            # value = [elem for elem in value[0].split(' ') if elem.strip()]
            # return value

            for col in value:
                col_split = col.split()

                # print(col_split[0])

                # extract the last element as the column name
                col_name = col_split[-1]

                new_col_name = col_split[0]

                ### if there is a "+" sign in the last element, concatenate the columns
                if "+" in col_name:
                    col_concat = col_name.split("+")
                    col_concat = [i.upper() for i in col_concat]

                    # concatenate the columns
                    df[new_col_name] = df.loc[:, col_concat].apply(lambda x: ''.join(['0' + str(i) if i < 10 else str(i) for i in x]), axis=1)

                else:
                    col_name = col_name.upper()
                    print(type(col_name))


def NEWVARIABLE_(data, df):
    for key, value in data.items():
        ### To read the file used
        if str('VARIABLE').lower() in str(key).lower():
            value = [s.split()[0] for s in value]
            
            for name in value:
                df[name] = 0


def TRANSFORM_(data, df):
    # Initialize empty arrays to store the values
    Arr1 = []
    Arr2 = []
    Arr3 = []

    for key, value in data.items():
        ### To read the file used
        if str('TRANSFORM').lower() in str(key).lower():
            lines = '\n'.join(value)
            lines = lines.split("\n")

            # Loop through the lines and extract the values
            for line in lines:
                line_values = line.split('=')  # split the line into values
                var_name = line_values[0].strip()  # extract the variable name
                var_value = line_values[1].strip().split()[0]  # extract the variable value
                condition = 'None'  # initialize the condition to None

                # Check if the line contains "Otherwise"
                if "Otherwise" in line:
                    condition = "Otherwise"

                # Check if the variable value contains an "in" statement
                if "in" in line:
                    if "NOT in" in line:
                        condition = line.split()[1] + ' ' + line.split()[2]
                    else:
                        condition = line.split()[1]

                # Append the values to the arrays
                Arr1.append(var_name)
                Arr2.append(var_value)
                Arr3.append(condition)

    return Arr1, Arr2, Arr3
        

def EST_RSE_(data):
    for key, value in data.items():
        pass

# print(FILE_(data))
df = pd.read_csv(FILE_(data))
df = df.rename(columns=lambda x: x.strip())

NEWVARIABLE_(data, df)

<ipython-input-6-3fe66c382919>:131: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(FILE_(data))


In [7]:
df.columns, df['POP']

(Index(['SEC', 'ST', 'FSU', 'IDNO', 'SEG', 'SSS', 'SSU', 'STRM', 'SSTRM', 'SS',
        'NSS', 'NSC', 'MULT', 'HH_SIZE', 'HHTYPE', 'RELG', 'SG', 'MHCE',
        'DC_ALL', 'DC_ST', 'SRL', 'REL', 'SEX', 'MSEX', 'AGE', 'MARST',
        'GEDU_LVL', 'PAS', 'PS_SS', 'CWS', 'ERN_SELF', 'ERN_REG', 'POP', 'LF',
        'WRK'],
       dtype='object'), 0         0
 1         0
 2         0
 3         0
 4         0
          ..
 109851    0
 109852    0
 109853    0
 109854    0
 109855    0
 Name: POP, Length: 109856, dtype: int64)

In [8]:
# Define the input string
input_str = '''ST_GR="G1"   in(ST,"10,19,20,21")
                  ST_GR="G2"   in(ST,"11,12,13,14,15,16,17,18")
                  ST_GR="G3"   Otherwise
                  POP=1
                  WRK=1  in(CWS,"11,12,21,31,41,51,61,61,71,72")
                  WRK=0  NOT in(CWS,"11,12,21,31,41,51,61,61,71,72")
                  LF=1      in(CWS,"11,12,21,31,41,51,61,61,71,72,81")
                  LF=0      Otherwise'''

# Split the input string into lines
lines = input_str.split('\n')

# Initialize empty arrays to store the values
Arr1 = []
Arr2 = []
Arr3 = []

# Loop through the lines and extract the values
for line in lines:
    line_values = line.split('=')  # split the line into values
    var_name = line_values[0].strip()  # extract the variable name
    var_value = line_values[1].strip().split()[0]  # extract the variable value
    condition = 'None'  # initialize the condition to None

    # Check if the line contains "Otherwise"
    if "Otherwise" in line:
        condition = "Otherwise"

    # Check if the variable value contains an "in" statement
    if "in" in line:
        if "NOT in" in line:
            condition = line.split()[1] + ' ' + line.split()[2]
        else:
            condition = line.split()[1]

    # Append the values to the arrays
    Arr1.append(var_name)
    Arr2.append(var_value)
    Arr3.append(condition)

# Print the results
# print(Arr1)
# print(Arr2)
# print(Arr3)

Arr4 = []
Arr5 = []

for i in range(len(Arr3)):
    if 'in' in Arr3[i]:
        if 'NOT in' in Arr3[i]:
            # pattern
            pattern = r'^NOT\s+in\((\w+),\s*"([^"]+)"\)$'
            # match
            match = re.match(pattern, Arr3[i])

            Arr4.append(match.group(1))
            Arr5.append(match.group(2))
            
        else:
            # pattern
            pattern = r'^in\((\w+),\s*"([^"]+)"\)$'
            # match
            match = re.match(pattern, Arr3[i])

            Arr4.append(match.group(1))
            Arr5.append(match.group(2))

    elif 'Otherwise' in Arr3[i]:
        # pattern
        pattern = r'^Otherwise$'
        # match
        match = re.match(pattern, Arr3[i])

        Arr4.append(match.group(0))
        Arr5.append(None)

    else:
        Arr4.append(None)
        Arr5.append(None)


In [9]:

lst = []
for value in Arr5:
    if value is None:
        lst.append(value)
    else:
        lst.append([int(x) for x in value.split(',')])

Arr5 = lst

In [10]:
set(Arr1) - set(list(df.columns))

{'ST_GR'}

In [11]:

# Check for name mismatches and create new columns
mismatched_names = set(Arr1) - set(df.columns)

new_columns = {name: [0] * df.shape[0] for name in mismatched_names}
df = df.assign(**new_columns)

mismatched_names

{'ST_GR'}

In [12]:
list(mismatched_names)

['ST_GR']

In [13]:
Arr5, Arr4, Arr3, Arr2, Arr1, df.columns

([[10, 19, 20, 21],
  [11, 12, 13, 14, 15, 16, 17, 18],
  None,
  None,
  [11, 12, 21, 31, 41, 51, 61, 61, 71, 72],
  [11, 12, 21, 31, 41, 51, 61, 61, 71, 72],
  [11, 12, 21, 31, 41, 51, 61, 61, 71, 72, 81],
  None],
 ['ST', 'ST', 'Otherwise', None, 'CWS', 'CWS', 'CWS', 'Otherwise'],
 ['in(ST,"10,19,20,21")',
  'in(ST,"11,12,13,14,15,16,17,18")',
  'Otherwise',
  'None',
  'in(CWS,"11,12,21,31,41,51,61,61,71,72")',
  'NOT in(CWS,"11,12,21,31,41,51,61,61,71,72")',
  'in(CWS,"11,12,21,31,41,51,61,61,71,72,81")',
  'Otherwise'],
 ['"G1"', '"G2"', '"G3"', '1', '1', '0', '1', '0'],
 ['ST_GR', 'ST_GR', 'ST_GR', 'POP', 'WRK', 'WRK', 'LF', 'LF'],
 Index(['SEC', 'ST', 'FSU', 'IDNO', 'SEG', 'SSS', 'SSU', 'STRM', 'SSTRM', 'SS',
        'NSS', 'NSC', 'MULT', 'HH_SIZE', 'HHTYPE', 'RELG', 'SG', 'MHCE',
        'DC_ALL', 'DC_ST', 'SRL', 'REL', 'SEX', 'MSEX', 'AGE', 'MARST',
        'GEDU_LVL', 'PAS', 'PS_SS', 'CWS', 'ERN_SELF', 'ERN_REG', 'POP', 'LF',
        'WRK', 'ST_GR'],
       dtype='object'))

In [87]:
for i in range(len(Arr3)):

    if 'in' in Arr3[i]:
        curr_column = 0
        lst_stack = []

        ### For NOT in case 
        if 'NOT in' in Arr3[i]:
            if Arr1[i] in list(df.columns):
                df.loc[~df[Arr4[i]].isin(Arr5[i]), (Arr1[i])] = Arr2[i]
            else:
                pass

        ### For in case
        else:
            if Arr1[i] in list(df.columns) and Arr5[i] is not None:
                # print(Arr1[i], Arr2[i], Arr4[i], Arr5[i])
                print(i)
                df.loc[df[Arr4[i]].isin(Arr5[i]), (Arr1[i])] = Arr2[i]
                lst_stack.extend(Arr5[i])
                curr_column = Arr4[i]
                break

            else:
                pass

    elif 'Otherwise' in Arr3[i]:
            if Arr1[i] in list(df.columns) and curr_column == True:
                ### Otherwise case (11,12,13,14,15,16,17,18,10,19,20,21)
                df.loc[~df[curr_column].isin(lst_stack), (Arr1[i])] = Arr2[i]
            else:
                pass

    else:
        if Arr1[i] in list(df.columns):
            # if Arr5[i] == None or Arr5[i] == 'None':
            df.loc[df[Arr1[i]] == 0, Arr1[i]] = Arr2[i]

    # print(lst_stack)


0


In [92]:
df[''].unique()

array(['1'], dtype=object)

In [33]:
# Arr5, Arr4, Arr3, Arr2, Arr1

Arr4, Arr3, Arr1, Arr5

(['ST', 'ST', 'Otherwise', None, 'CWS', 'CWS', 'CWS', 'Otherwise'],
 ['in(ST,"10,19,20,21")',
  'in(ST,"11,12,13,14,15,16,17,18")',
  'Otherwise',
  'None',
  'in(CWS,"11,12,21,31,41,51,61,61,71,72")',
  'NOT in(CWS,"11,12,21,31,41,51,61,61,71,72")',
  'in(CWS,"11,12,21,31,41,51,61,61,71,72,81")',
  'Otherwise'],
 ['ST_GR', 'ST_GR', 'ST_GR', 'POP', 'WRK', 'WRK', 'LF', 'LF'],
 [[10, 19, 20, 21],
  [11, 12, 13, 14, 15, 16, 17, 18],
  None,
  None,
  [11, 12, 21, 31, 41, 51, 61, 61, 71, 72],
  [11, 12, 21, 31, 41, 51, 61, 61, 71, 72],
  [11, 12, 21, 31, 41, 51, 61, 61, 71, 72, 81],
  None])

In [ ]:
# Check for name mismatches and create new columns
mismatched_names = set(Arr1) - set(df.columns)
new_columns = {name: [0] * df.shape[0] for name in mismatched_names}
df = df.assign(**new_columns)

for i in range(len(Arr1)):
    if Arr3[i] == 'in':
        if isinstance(Arr5[i], int):
            flattened_list = [Arr5[i]]
        else:
            flattened_list = [item for sublist in Arr5[i] for item in sublist] if (isinstance(Arr5[i], list) or isinstance(Arr5[i], tuple)) else [Arr5[i]]
        df.loc[df[Arr4[i]].isin(flattened_list), (Arr1[i])] = Arr2[i]
    elif Arr3[i] == 'NOT in':
        if isinstance(Arr5[i], int):
            flattened_list = [Arr5[i]]
        else:
            flattened_list = [item for sublist in Arr5[i] for item in sublist] if (isinstance(Arr5[i], list) or isinstance(Arr5[i], tuple)) else [Arr5[i]]
        df.loc[~df[Arr4[i]].isin(flattened_list), (Arr1[i])] = 0


In [ ]:
df['ST_GR'].unique()

In [ ]:
df.columns

In [ ]:
ldsdsist(set(Arr1)), Arr1, Arr2, Arr4, Arr5

# # check for name mismatches and create new columns
# mismatched_names = set(Arr1) - set(list(df.columns))
# new_columns = {name: [0]*df.shape[0] for name in mismatched_names}
# df = df.assign(**new_columns)
# df.columns

# for k, v in new_columns.items():
#     print(k)
#     df.loc[df[new_columns].isin([5, 6]), ('newVar1')] = df['    ERN_REG'] + df['   ERN_SELF']
#     df.loc[~df['  GEDU_LVL'].isin([5, 6]), ('newVar1')] = 0

In [ ]:
print(new_columns)

In [ ]:

# df['newVar1'] = pd.Series([])
# df.loc[df['  GEDU_LVL'].isin([5, 6]), ('newVar1')] = df['    ERN_REG'] + df['   ERN_SELF']
# df.loc[~df['  GEDU_LVL'].isin([5, 6]), ('newVar1')] = 0

In [ ]:
import re

# input string
string = 'NOT in(CWS,"11,12,21,31,41,51,61,61,71,72")'

# pattern
pattern = r'^NOT\s+in\((\w+),\s*"([^"]+)"\)$'

# match
match = re.match(pattern, string)

print(match.group(1), type(match.group(2)))

# extract groups
if match:
    cws = match.group(1)
    integers = match.group(2)
    integers_list = integers.split(',')
    integers_set = set(integers_list)
    print(f"CWS: {cws}, Integers: {integers_set}")
else:
    print("No match found.")


# convert integers to set of integers
integers_set = set(map(int, integers_list))
print(f"CWS: {cws}, Integers: {integers_set}")
integers_set

In [ ]:
### Reading and filtering column names
df = pd.read_csv(FILE_(data))
df = df.rename(columns=lambda x: x.strip())

# print(NEWVARIABLE_(data, df)[0])

for col in RENAME_(data, df):
    col = [elem for elem in col.split(' ') if elem.strip()]
    
    # df[col[0]] = df[]

    col_search = col[2].split('+')

    # print(col_search)

    for txt in col_search:
        if txt in df.columns.str.lower():
            pass
            
        # else:
        #     print('No')

    # print(col[2].split('+'))

# df.query(FILTER_(data))

# df = pd.read_csv(FILE_(data))
# df.head(2)

In [ ]:
# ## read the RSE file and extract the lines from it
# with open('Example1.RSE') as f:

#     ## Reading each line of the file 
#     lines = f.readlines()

# ## To generate keys of the dictionary 
# keys = []

# for line in lines:
#     # print(line)
#     line = line.strip()

#     if line.startswith("#"):

#         quote_index = line.find("'") 
#         if quote_index != -1:
#             key = line[1:quote_index].split()

#             # Check if the line is empty
#             if not line:
#                 # Empty line found, break out of the loop
#                 break

#             if key not in keys:
#                 keys.append(key)

# print(keys)

In [ ]:
### renaming the variables
### **********************

# rename the columns using a dictionary
new_columns = {
    'FSU': 'fsu',
    'NS_SSS': 'NSS'
}

df = df.rename(columns=new_columns)

df['STRMID'] = df['SEC'].apply(lambda x: f'0{x}' if x < 10 else str(x)) + \
                df['AST'].apply(lambda x: f'0{x}' if x < 10 else str(x)) + \
                 df[' STRM'].apply(lambda x: f'0{x}' if x < 10 else str(x)) + \
                  df['  SSTRM'].apply(lambda x: f'0{x}' if x < 10 else str(x))

df[' SSU'] = df[' FSU'].astype(str) + df[' IDNO'].apply(lambda x: f'0{x}' if x < 10 else str(x))

In [ ]:
df[' SSU']

In [ ]:
age = df['  AGE']

# Function to calculate the right age group
def get_age_group(age):
    if age < 5:
        return "01"
    elif age >= 5 and age < 10:
        return "02"
    elif age >= 10 and age < 20:
        return "03"
    elif age >= 20 and age < 60:
        return "04"
    elif age >= 60:
        return "05"
    else:
        return "00"

# Apply the get_age_group function to each row in the "age" column in parallel
df['AGE_GROUP'] = df['  AGE'].parallel_apply(get_age_group)


In [ ]:
### Age group with right age
def get_result(df):
    age = df['  AGE']
    ageGrp = df['AGE_GROUP']

    if ageGrp == "01" and age < 5:
        return "01"
    elif ageGrp == "02" and age >= 5 and age < 10:
        return "02"
    elif ageGrp == "03" and age >= 10 and age < 20:
        return "03"
    elif ageGrp == "04" and age >= 20 and age < 60:
        return "04"
    elif ageGrp == "05" and age>=60:
        return "00"
    else:
        return "00"

df['AGE_GROUP'] = df.parallel_apply(get_result, axis=1)

In [ ]:
df.columns

In [ ]:
def Y1_(df):
    df['newVar1'] = pd.Series([])
    df.loc[df['  GEDU_LVL'].isin([5, 6]), ('newVar1')] = df['    ERN_REG'] + df['   ERN_SELF']
    df.loc[~df['  GEDU_LVL'].isin([5, 6]), ('newVar1')] = 0

def Y2_(df):
    df['newVar2'] = pd.Series([])
    df.loc[df['  GEDU_LVL'].isin([7, 8, 9]), ('newVar2')] = df['    ERN_REG'] + df['   ERN_SELF']
    df.loc[~df['  GEDU_LVL'].isin([7, 8, 9]), ('newVar2')] = 0

def Y3_(df):
    df['newVar3'] = pd.Series([])
    df.loc[df['  GEDU_LVL'].isin([5, 6, 7, 8, 9]), ('newVar3')] = df['    ERN_REG'] + df['   ERN_SELF']
    df.loc[~df['  GEDU_LVL'].isin([5, 6, 7, 8, 9]), ('newVar3')] = 0

def Y4_(df):
    df['newVar4'] = pd.Series([])
    df.loc[df['   ACWS'].isin([11,12,21,31,41,51,61,61,71,72]), ('newVar4')] = 1
    df.loc[~df['   ACWS'].isin([11,12,21,31,41,51,61,61,71,72]), ('newVar4')] = 0

def Y5_(df):
    df['newVar5'] = pd.Series([])
    df.loc[df['   ACWS'].isin([11,12,21,31,41,51,61,61,71,72,81]), ('newVar5')] = 1
    df.loc[~df['   ACWS'].isin([11,12,21,31,41,51,61,61,71,72,81]), ('newVar5')] = 0

def Y6_(df):
    df['newVar6'] = 1

In [ ]:
Y1_(df), Y2_(df), Y3_(df), Y4_(df), Y5_(df), Y6_(df)

In [ ]:
# ### Example-2: Defining another variable

# print("select number of variables : ")

# n_var = int(input())

# ### creating no. of n_var
# for i in range(1, n_var+1):
#     # df['Y' + str(i)] = pd.Series([])
#     df['X' + str(i)] = pd.Series([])

#     if i == 1:
#         df.loc[df['  GEDU_LVL'].isin([5, 6]), ('X' + str(i))] = df['    ERN_REG'] + df['   ERN_SELF']
#         df.loc[~df['  GEDU_LVL'].isin([5, 6]), ('X' + str(i))] = 0
#     elif i == 2:
#         df.loc[df['  GEDU_LVL'].isin([7, 8, 9]), ('X' + str(i))] = df['    ERN_REG'] + df['   ERN_SELF']
#         df.loc[~df['  GEDU_LVL'].isin([5, 6]), ('X' + str(i))] = 0
#     elif i == 3:
#         df.loc[df['  GEDU_LVL'].isin([5, 6, 7, 8, 9]), ('X' + str(i))] = df['    ERN_REG'] + df['   ERN_SELF']
#         df.loc[~df['  GEDU_LVL'].isin([5, 6]), ('X' + str(i))] = 0
    

In [ ]:
df['newVar3'].unique()

In [ ]:
### Transform the variables



In [ ]:
df.to_csv('GROUP SEC ST AGE_Group.csv')

In [ ]:
df